In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

In [2]:
# Define RMSE as a custom metric
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Load the data
file_path = 'cleaned.csv'  # Replace with your actual path
data = pd.read_csv(file_path, low_memory=False)
data.head()

,epoch,startTimestamp,lockTimestamp,closeTimestamp,lockPrice,closePrice,totalAmount,bullAmount,bearAmount,rewardBaseCalAmount,rewardAmount,2round_price_increased
0,1,1629927751,1629928051,1629928363,50081000000,50027000000,0.0,0.0,0.0,0.0,0.0,True
1,2,1629928063,1629928363,1629928672,50027000000,50208000000,0.0,0.0,0.0,0.0,0.0,False
2,3,1629928372,1629928672,1629928984,50208000000,50399415626,0.0,0.0,0.0,0.0,0.0,True
3,4,1629928684,1629928984,1629929293,50399415626,50143000000,0.0,0.0,0.0,0.0,0.0,False
4,5,1629928993,1629929293,1629929602,50143000000,50222000000,0.0,0.0,0.0,0.0,0.0,False


In [3]:
# Preprocess the data
# Drop unnecessary columns if needed (e.g., 'epoch', timestamps if not useful)
X = data.drop(columns=['closePrice', 'epoch', 'startTimestamp', 'lockTimestamp', 'closeTimestamp', '2round_price_increased'])
y = data['closePrice']

# Convert scientific notation strings to floats
X = X.apply(pd.to_numeric, errors='coerce')
y = pd.to_numeric(y, errors='coerce')

# Fill or drop missing values if present
X.fillna(0, inplace=True)  # Fill with 0 or use X.dropna(inplace=True)
y.fillna(0, inplace=True)

In [4]:
# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, shuffle=False)

# Build the neural network model
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')  # Output layer for regression
])

# Compile the model with RMSE as a metric
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error', rmse])

In [5]:
# Train the model
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=256, verbose=1)

Epoch 1/50
904/904 [==============================] - 7s 5ms/step - loss: 1182684950119208452096.0000 - mean_absolute_error: 32968867840.0000 - rmse: 34382835712.0000 - val_loss: 1692538308358752436224.0000 - val_mean_absolute_error: 38588076032.0000 - val_rmse: 38603030528.0000
Epoch 2/50
904/904 [==============================] - 4s 4ms/step - loss: 1182681290944511213568.0000 - mean_absolute_error: 32968812544.0000 - rmse: 34383765504.0000 - val_loss: 1692533241809171644416.0000 - val_mean_absolute_error: 38587985920.0000 - val_rmse: 38602960896.0000
Epoch 3/50
904/904 [==============================] - 4s 4ms/step - loss: 1182668061620605812736.0000 - mean_absolute_error: 32968642560.0000 - rmse: 34383011840.0000 - val_loss: 1692521419860149796864.0000 - val_mean_absolute_error: 38587850752.0000 - val_rmse: 38602780672.0000
Epoch 4/50
904/904 [==============================] - 4s 4ms/step - loss: 1182645121410003894272.0000 - mean_absolute_error: 32968290304.0000 - rmse: 3438363852

In [7]:
# Evaluate the model
loss, mae, rmse_value = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Mean Absolute Error: {mae/1e8}")
print(f"Test RMSE: {rmse_value/1e8}")

2259/2259 [==============================] - 6s 3ms/step - loss: 3663194722855909588992.0000 - mean_absolute_error: 60166918144.0000 - rmse: 60173877248.0000
Test Mean Absolute Error: 601.66918144
Test RMSE: 601.73877248
